In [2]:
import numpy as np
import matplotlib.pyplot as plt
import phasepy as pp

In [15]:
def simulacion(initial_conditions):
    # Desempaquetado de condiciones iniciales
    Ti, Pi = initial_conditions
    
    # Datos del nitrógeno obtenidos del Perry's Handbook
    nitrogen = pp.component(name = 'nitrogen', Tc = 162.2, Pc = 3.39e2, Zc = 0.288, Vc = 89, w = 0.037)
    oxygen = pp.component(name = 'oxygen', Tc = 154.58, Pc = 5.02e2, Zc = 0.287, Vc = 74, w = 0.02)
    
    # Modelado de compuestos con Peng-Robinson
    eos_nitrogeno = pp.preos(nitrogen)
    eos_oxygen = pp.preos(oxygen)

    # Cálculo del volumen inicial a Ti y Pi
    volume_nitrogen_i = 1 / eos_nitrogeno.density(Ti, Pi, state='V')
    print(volume_nitrogen_i)

    # Isothermal compression
    

In [16]:
# Initial conditions:
Ti, Pi = 25 + 273.15, 1.01325
initial_conditions = [Ti, Pi]

simulacion(initial_conditions)

24460.04765977195


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Definición de funciones
def choose(c1, c2, c3, c4, pick):
    return [c1, c2, c3, c4][pick - 1]

def adiW(t, T1, Cv):
    return Cv * (t - T1) / 1000

def plot_expansion_compression(bn, pick1, pick2, go, Pf):
    # Cálculo
    P1 = 0.1 * 10*6 if bn == 1 else 1 * 10*6
    P2 = Pf * 10**6

    R = 8.314
    T1 = 300
    V1 = 1000 * R * T1 / P1

    # Isotérmico
    isoV2 = 1000 * R * T1 / P2
    isoWR = -R * T1 * np.log(isoV2 / V1) / 1000
    isoWX = -P2 * (isoV2 - V1) / 1000 / 1000

    # Adiabático
    gamma = 7 / 5
    Cv = 5 * R / 2
    adiV2R = V1 * (P1 / P2)(1 / gamma)
    adiV2X = 1000 * (R * (Cv * T1 + P2 * V1 / 1000)) / (P2 * (Cv + R))
    T2R = T1 * (V1 / adiV2R)(gamma - 1)
    T2X = T1 - P2 * (adiV2X - V1) / 1000 / Cv

    v1 = V1 + (choose(isoV2, adiV2R, isoV2, adiV2X, pick1) - V1) * go
    v2 = V1 + (choose(isoV2, adiV2R, isoV2, adiV2X, pick2) - V1) * go

    Pint = (P1 + (P2 - P1) * go) / 10**6
    Pext1 = Pint if pick1 in [1, 2] else P2 / 10**6
    Pext2 = Pint if pick2 in [1, 2] else P2 / 10**6
    Pf = P2 / 10**6

    wb1 = go if bn == 1 else (0.9 - (1 - Pf) * go if pick1 in [1, 2] else Pf / 1.1)
    wt1 = (go * (Pf - 1.1) / (2 - 1.1) if bn == 1 else (1 - go if pick1 in [1, 2] else 0))

    wb2 = go if bn == 1 else (0.9 - (1 - Pf) * go if pick2 in [1, 2] else Pf / 1.1)
    wt2 = (go * (Pf - 1.1) / (2 - 1.1) if bn == 1 else (1 - go if pick2 in [1, 2] else 0))

    # Graficación
    fig, ax = plt.subplots()
    delta_x, delta_w, delta_h, th, height = 10, 2, 1, 0.5, 1

    rect1 = plt.Rectangle((0, 0), delta_x, v1, color='green', alpha=0.7)
    rect2 = plt.Rectangle((delta_x + delta_w, 0), delta_x, v2, color='green', alpha=0.7)
    rect3 = plt.Rectangle((0, v1), delta_x, 1.5, edgecolor='gray', facecolor='gray', linewidth=3)
    rect4 = plt.Rectangle((delta_x + delta_w, v2), delta_x, 1.5, edgecolor='gray', facecolor='gray', linewidth=3)

    ax.add_patch(rect1)
    ax.add_patch(rect2)
    ax.add_patch(rect3)
    ax.add_patch(rect4)

    plt.xlim(-1, 2 * delta_x + delta_w + 1)
    plt.ylim(-1, max(v1, v2) + 2.5)

    ax.text(delta_x / 2, 0.5 * v1, f"P = {Pint:.2f} MPa", fontsize=12, ha='center')
    ax.text(1.5 * delta_x + delta_w, 0.5 * v2, f"P = {Pint:.2f} MPa", fontsize=12, ha='center')

    ax.text(delta_x / 2, v1 + 1, f"Pext = {Pext1:.2f} MPa", fontsize=12, ha='center')
    ax.text(1.5 * delta_x + delta_w, v2 + 1, f"Pext = {Pext2:.2f} MPa", fontsize=12, ha='center')

    plt.title("Reversible and Irreversible Expansion or Compression Work")
    plt.xlabel("Volume")
    plt.ylabel("Pressure")
    plt.grid(False)
    plt.show()

    # Widgets interactivos
interact(plot_expansion_compression,
         bn=widgets.ToggleButtons(options=[('Compression', 1), ('Expansion', 2)], description='Process:'),
         pick1=widgets.Dropdown(options=[('Reversible Isothermal', 1), ('Reversible Adiabatic', 2), ('Irreversible Isothermal', 3), ('Irreversible Adiabatic', 4)], description='Condition 1:'),
         pick2=widgets.Dropdown(options=[('Reversible Isothermal', 1), ('Reversible Adiabatic', 2), ('Irreversible Isothermal', 3), ('Irreversible Adiabatic', 4)], description='Condition 2:'),
         go=widgets.FloatSlider(value=0, min=0, max=1, step=0.01, description='Progress:'),
         Pf=widgets.FloatSlider(value=1.5, min=0.1, max=2.0, step=0.1, description='Final Pressure (MPa):'))
